In [ ]:
!pip install scikit-learn
!pip install -U sentence-transformers
!pip install pinecone
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install --upgrade google-cloud-bigquery


  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 23.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 14.7 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
import os
project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
print(f"Your Colab Project ID is: {project_id}")


Authenticated
Your Colab Project ID is: None


In [ ]:
import pandas as pd
from google.cloud import bigquery
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import os

project_id = 'master-anagram-430800-f0'

if not project_id:
    raise ValueError("Google Cloud Project ID not set. Please provide your project ID.")
else:
    print(f"Using Google Cloud Project ID: {project_id}")

client = bigquery.Client(project=project_id)

# Modified queries dictionary with LIMIT 20 added to each query
queries = {
    'age': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.age` LIMIT 20",
    'acei': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.acei` LIMIT 20",
    'invasive_line': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.invasive_line` LIMIT 20",
    'kdigo_creatinine': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.kdigo_creatinine` LIMIT 20",
    'kdigo_stages': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.kdigo_stages` LIMIT 20",
    'kdigo_uo': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.kdigo_uo` LIMIT 20",
    'lods': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.lods` LIMIT 20",
    'meld': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.meld` LIMIT 20",
    'milrinone': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.milrinone` LIMIT 20",
    'neuroblock': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.neuroblock` LIMIT 20",
    'norepinephrine': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.norepinephrine` LIMIT 20",
    'norepinephrine_equivalent_dose': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.norepinephrine_equivalent_dose` LIMIT 20",
    'nsaid': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.nsaid` LIMIT 20",
    'oasis': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.oasis` LIMIT 20",
    'oxygen_delivery': "SELECT * FROM `physionet-data.mimiciv_3_1_derived.oxygen_delivery` LIMIT 20"
}

dataframes = {}

for table_name, query_string in queries.items():
    print(f"\nExecuting query for table: {table_name}")
    try:
        query_job = client.query(query_string)
        df = query_job.to_dataframe()
        dataframes[table_name] = df
        print(f"Successfully loaded data for {table_name} ({len(df)} rows).")
    except Exception as e:
        print(f"Error querying table {table_name}: {e}")
        print(f"Could not load data for {table_name}.")

Using Google Cloud Project ID: master-anagram-430800-f0

Executing query for table: age
Successfully loaded data for age (20 rows).

Executing query for table: acei
Successfully loaded data for acei (20 rows).

Executing query for table: invasive_line
Successfully loaded data for invasive_line (20 rows).

Executing query for table: kdigo_creatinine
Successfully loaded data for kdigo_creatinine (20 rows).

Executing query for table: kdigo_stages
Successfully loaded data for kdigo_stages (20 rows).

Executing query for table: kdigo_uo
Successfully loaded data for kdigo_uo (20 rows).

Executing query for table: lods
Successfully loaded data for lods (20 rows).

Executing query for table: meld
Successfully loaded data for meld (20 rows).

Executing query for table: milrinone
Successfully loaded data for milrinone (20 rows).

Executing query for table: neuroblock
Successfully loaded data for neuroblock (20 rows).

Executing query for table: norepinephrine
Successfully loaded data for norepi

In [ ]:
import pandas as pd
from google.cloud import bigquery
import os

print("\n--- Print the head of each loaded data frame")
if not dataframes:
  print("No tables ind dataframe")
else:
  for table_name, df in dataframes.items():
    print(f"\nHead of {table_name}:")
    print(df.head())


--- Print the head of each loaded data frame

Head of age:
   subject_id   hadm_id           admittime  anchor_age  anchor_year  age
0    13715714  25187061 2110-12-30 13:52:00          18         2110   18
1    10686175  27744664 2110-08-04 18:16:00          18         2110   18
2    17662180  24714720 2110-07-22 07:15:00          18         2110   18
3    15771780  27129872 2110-01-20 21:20:00          18         2110   18
4    14664573  28635572 2110-04-04 10:33:00          18         2110   18

Head of acei:
   subject_id   hadm_id             acei           starttime  \
0    13963934  29023975  *NF* Benazepril 2122-07-27 01:00:00   
1    10180204  21636925       Benazepril 2147-06-24 08:00:00   
2    10180204  21636925       Benazepril 2147-06-24 08:00:00   
3    10415954  28505443       Benazepril 2186-07-03 20:00:00   
4    10501780  23869567       Benazepril 2121-06-15 10:00:00   

             stoptime  
0 2122-07-27 17:00:00  
1 2147-06-25 23:00:00  
2 2147-06-24 03:00:00  


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pinecone import Pinecone, ServerlessSpec
import os
import numpy as np

def create_state_vector_all_features(df_dict):
    state_vectors = {}
    for stay_id in get_all_stay_ids(df_dict):
        stay_data = {}
        age_df = df_dict.get('age')
        if age_df is not None:
            patient_ages = age_df[age_df['subject_id'].isin(get_subject_ids_for_stay(df_dict, stay_id))]
            if not patient_ages.empty:
                stay_data['age'] = patient_ages['age'].iloc[0]
        kdigo_creat_df = df_dict.get('kdigo_creatinine')
        if kdigo_creat_df is not None:
            creat_value = kdigo_creat_df[kdigo_creat_df['stay_id'] == stay_id]['creat'].dropna().iloc[0] if not kdigo_creat_df[kdigo_creat_df['stay_id'] == stay_id]['creat'].dropna().empty else None
            if creat_value is not None:
                stay_data['creat'] = creat_value
        kdigo_stages_df = df_dict.get('kdigo_stages')
        if kdigo_stages_df is not None:
            aki_stage = kdigo_stages_df[kdigo_stages_df['stay_id'] == stay_id]['aki_stage'].dropna().iloc[0] if not kdigo_stages_df[kdigo_stages_df['stay_id'] == stay_id]['aki_stage'].dropna().empty else None
            if aki_stage is not None:
                try:
                    stay_data['aki_stage'] = float(aki_stage)
                except ValueError:
                    stay_data['aki_stage'] = np.nan
        kdigo_uo_df = df_dict.get('kdigo_uo')
        if kdigo_uo_df is not None:
            uo_24hr = kdigo_uo_df[kdigo_uo_df['stay_id'] == stay_id]['uo_rt_24hr'].dropna().iloc[0] if not kdigo_uo_df[kdigo_uo_df['stay_id'] == stay_id]['uo_rt_24hr'].dropna().empty else None
            if uo_24hr is not None:
                try:
                    stay_data['uo_rt_24hr'] = float(uo_24hr)
                except ValueError:
                    stay_data['uo_rt_24hr'] = np.nan
        lods_df = df_dict.get('lods')
        if lods_df is not None:
            lods_score = lods_df[lods_df['stay_id'] == stay_id]['lods'].dropna().iloc[0] if not lods_df[lods_df['stay_id'] == stay_id]['lods'].dropna().empty else None
            if lods_score is not None:
                try:
                    stay_data['lods'] = float(lods_score)
                except ValueError:
                    stay_data['lods'] = np.nan
            for col in ['neurologic', 'cardiovascular', 'renal', 'pulmonary', 'hematologic', 'hepatic']:
                value = lods_df[lods_df['stay_id'] == stay_id][col].dropna().iloc[0] if not lods_df[lods_df['stay_id'] == stay_id][col].dropna().empty else None
                if value is not None:
                    try:
                        stay_data[f'lods_{col}'] = float(value)
                    except ValueError:
                        stay_data[f'lods_{col}'] = np.nan
        oasis_df = df_dict.get('oasis')
        if oasis_df is not None:
            oasis_score = oasis_df[oasis_df['stay_id'] == stay_id]['oasis'].dropna().iloc[0] if not oasis_df[oasis_df['stay_id'] == stay_id]['oasis'].dropna().empty else None
            if oasis_score is not None:
                try:
                    stay_data['oasis'] = float(oasis_score)
                except ValueError:
                    stay_data['oasis'] = np.nan
            for col in ['gcs', 'resprate', 'temp', 'urineoutput']:
                value = oasis_df[oasis_df['stay_id'] == stay_id][col].dropna().iloc[0] if not oasis_df[oasis_df['stay_id'] == stay_id][col].dropna().empty else None
                if value is not None:
                    try:
                        stay_data[f'oasis_{col}'] = float(value)
                    except ValueError:
                        stay_data[f'oasis_{col}'] = np.nan
        if stay_data:
            state_vectors[stay_id] = stay_data
    return pd.DataFrame.from_dict(state_vectors, orient='index')

def get_all_stay_ids(df_dict):
    stay_ids = set()
    for df in df_dict.values():
        if 'stay_id' in df.columns:
            stay_ids.update(df['stay_id'].unique())
    return list(stay_ids)

def get_subject_ids_for_stay(df_dict, stay_id):
    subject_ids = set()
    for df in df_dict.values():
        if 'stay_id' in df.columns and 'subject_id' in df.columns:
            subject_ids.update(df[df['stay_id'] == stay_id]['subject_id'].unique())
        elif 'subject_id' in df.columns and 'hadm_id' in df.columns:
            pass
    for df in df_dict.values():
        if 'stay_id' in df.columns and 'subject_id' in df.columns and not df[df['stay_id'] == stay_id]['subject_id'].empty:
            return df[df['stay_id'] == stay_id]['subject_id'].unique()
        elif 'hadm_id' in df.columns and 'subject_id' in df.columns:
            pass
    return set()

patient_states_all_df = create_state_vector_all_features(dataframes)

if not patient_states_all_df.empty:
    patient_states_imputed_df = patient_states_all_df.fillna(patient_states_all_df.mean(numeric_only=True)).fillna(0)
    scaler_all = StandardScaler()
    scaled_states_all = scaler_all.fit_transform(patient_states_imputed_df)
    scaled_states_all_df = pd.DataFrame(scaled_states_all, index=patient_states_imputed_df.index, columns=patient_states_imputed_df.columns)

    vectors_for_pinecone = []
    for stay_id, row in scaled_states_all_df.iterrows():
        vector = row.tolist()
        print(f"Stay ID: {stay_id}, Vector (before adding to upsert): {vector}") # ADDED PRINT STATEMENT
        vectors_for_pinecone.append((str(stay_id), vector))

    print("\nScaled state vectors prepared for Pinecone (first 5):")
    for i in range(min(5, len(vectors_for_pinecone))):
        print(f"ID: {vectors_for_pinecone[i][0]}, Vector: {vectors_for_pinecone[i][1]}")

else:
    print("\nNo patient state vectors could be created.")

Stay ID: 30883841, Vector (before adding to upsert): [-1.943189559225357, 0.47572659109372734, 0.7247162041387062, 0.9966978520003051, -0.9732666244212825, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Stay ID: 34938899, Vector (before adding to upsert): [-0.8327955253822958, 0.47572659109372734, 0.4096222023392688, -1.7829375918252917, 0.8332020504170127, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Stay ID: 36298295, Vector (before adding to upsert): [-1.943189559225357, 0.47572659109372734, -1.7960358102567928, -0.009265451479425302, -2.5130661139263055, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Stay ID: 30257222, Vector (before adding to upsert): [-1.943189559225357, 0.47572659109372734, -3.686599821053417, 3.80280601433854, -1.016277783346004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Stay ID: 34183312, Vector (before adding to upsert): [1.3879925423038264, 0.47572659109372734, -3.686599821053417, -1.1873014252912297, -3.2528580474315123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.03961718  0.13254842 -0.04268068 ... -0.07988974 -0.08624619
   0.06385779]
 [ 0.03305054  0.00307323 -0.04791299 ... -0.02800244 -0.07863103
   0.09365849]
 [ 0.05315324  0.00699715 -0.01184396 ...  0.04102099 -0.04138892
   0.01170313]
 ...
 [ 0.00105446  0.06201827 -0.00179886 ... -0.03452431  0.02412905
  -0.00441833]
 [ 0.10959973  0.03775932 -0.01546354 ... -0.02857913  0.00059377
   0.00079295]
 [ 0.00512687 -0.0143538  -0.04267336 ...  0.02397272 -0.00917198
  -0.00131434]]


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="08180080-18aa-4999-b131-612b5b9b41aa")
index_name = "ucsfresearch"

try:
    if pc.has_index(index_name):
        pc.delete_index(index_name)
        print(f"Deleted Pinecone index '{index_name}'.")
    else:
        print(f"Index '{index_name}' does not exist.")
except Exception as e:
    print(f"Error deleting index: {e}")

Deleted Pinecone index 'ucsfresearch'.


In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os
import numpy as np

def safe_meta(value, fallback):
    return fallback if value is None else value

if 'scaled_states_all_df' in locals() and not scaled_states_all_df.empty:
    pc = Pinecone(api_key="08180080-18aa-4999-b131-612b5b9b41aa")
    index_name = "ucsfresearch"
    vector_dimension = scaled_states_all_df.shape[1]

    try:
        if not pc.has_index(index_name):
            print(f"Creating index '{index_name}' with dimension {vector_dimension}...")
            pc.create_index(
                name=index_name,
                dimension=vector_dimension,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1")
            )
            print(f"Index '{index_name}' created.")
        else:
            print(f"Index '{index_name}' exists.")
            existing_index_info = pc.describe_index(index_name)
            if existing_index_info.dimension != vector_dimension:
                print(f"WARNING: Index dimension ({existing_index_info.dimension}) does not match vector dimension ({vector_dimension}).")

        index = pc.Index(index_name)

        vectors_to_upsert = []
        epsilon = 1e-9  # Threshold for non-zero vectors

        for stay_id, row in scaled_states_all_df.iterrows():
            vector = np.array(row.tolist())

            if np.any(np.abs(vector) > epsilon):
                # Replace with actual sources if these columns are from another DataFrame
                subject_id = row.get("subject_id", "unknown")
                treatment = row.get("treatment", "unknown")
                mortality = row.get("mortality_30d", False)

                vectors_to_upsert.append({
                    "id": str(stay_id),
                    "values": vector.tolist(),
                    "metadata": {
                        "subject_id": str(safe_meta(subject_id, "unknown")),
                        "treatment": str(safe_meta(treatment, "unknown")),
                        "mortality_30d": bool(safe_meta(mortality, False))
                    }
                })
            else:
                print(f"Skipping all-zero vector with ID: {stay_id} (values effectively zero)")

        if vectors_to_upsert:
            print(f"Upserting {len(vectors_to_upsert)} valid vectors...")
            index.upsert(
                vectors=vectors_to_upsert,
                namespace="data"
            )
            print(f"Upserted {len(vectors_to_upsert)} vectors.")
            print("\nIndex Stats:")
            print(index.describe_index_stats())
        else:
            print("No valid vectors to upsert (all were effectively all-zeros).")

    except Exception as e:
        print(f"\nPinecone error: {e}")
else:
    print("No scaled state vectors to upsert.")


Creating index 'ucsfresearch' with dimension 14...
Index 'ucsfresearch' created.
Skipping all-zero vector with ID: 39546424 (values effectively zero)
Skipping all-zero vector with ID: 33576525 (values effectively zero)
Skipping all-zero vector with ID: 38341713 (values effectively zero)
Skipping all-zero vector with ID: 39901293 (values effectively zero)
Skipping all-zero vector with ID: 38036594 (values effectively zero)
Skipping all-zero vector with ID: 39198840 (values effectively zero)
Skipping all-zero vector with ID: 31943823 (values effectively zero)
Skipping all-zero vector with ID: 38698126 (values effectively zero)
Skipping all-zero vector with ID: 38469361 (values effectively zero)
Skipping all-zero vector with ID: 38975229 (values effectively zero)
Skipping all-zero vector with ID: 38236938 (values effectively zero)
Skipping all-zero vector with ID: 36632875 (values effectively zero)
Skipping all-zero vector with ID: 34241944 (values effectively zero)
Skipping all-zero vect

In [ ]:
from pinecone import Pinecone
import numpy as np

pc = Pinecone(api_key="08180080-18aa-4999-b131-612b5b9b41aa")
index_name = "ucsfresearch"

def find_similar_patients(patient_state_vector, top_k=5):
    if len(patient_state_vector) != 14:
        print(f"Error: Input vector must have 14 dimensions, but got {len(patient_state_vector)}.")
        return None
    try:
        index = pc.Index(index_name)
        query_results = index.query(
            vector=patient_state_vector,
            namespace="data",
            top_k=top_k,
            include_metadata=True
        )
        return query_results.matches
    except Exception as e:
        print(f"Error querying Pinecone: {e}")
        return None

if 'scaled_states_all_df' in locals() and not scaled_states_all_df.empty:
    example_patient_vector = scaled_states_all_df.iloc[0].tolist()
    print("Example input patient state vector:", example_patient_vector)

    similar_patients = find_similar_patients(example_patient_vector, top_k=3)

    if similar_patients:
        print("\nTop 3 most similar patients:")
        for match in similar_patients:
            print(f"ID: {match.id}, Similarity Score: {match.score}")
    else:
        print("Could not retrieve similar patients.")
else:
    print("No scaled data available to create an example input vector.")

Example input patient state vector: [-1.943189559225357, 0.47572659109372734, 0.7247162041387062, 0.9966978520003051, -0.9732666244212825, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Top 3 most similar patients:
ID: 30883841, Similarity Score: 0.999752939
ID: 38663369, Similarity Score: 0.937669218
ID: 31552189, Similarity Score: 0.775052488


In [ ]:
print("\nIndex Stats:")
print(index.describe_index_stats())




Index Stats:
{'dimension': 14,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'data': {'vector_count': 66}},
 'total_vector_count': 66,
 'vector_type': 'dense'}


[100, np.float32(3.7109394), np.float32(2.7788718), np.float32(2.623132), np.float32(2.5300477), np.float32(2.319258), np.float32(2.277316), np.float32(2.1066942), np.float32(1.9946189), np.float32(1.9923733), np.float32(1.9227493), np.float32(1.9113404), np.float32(1.9044847), np.float32(1.7756509), np.float32(1.7484009), np.float32(1.7364461), np.float32(1.6589296), np.float32(1.588579), np.float32(1.5609161), np.float32(1.5298895)]


In [ ]:
# Making a Decision Tree(Random Forrest)
from sckitlearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import graphviz